In [1]:
import matplotlib.pyplot as plt
import time
import torch
from torch import logsumexp
import torch.nn.functional as F
from torch.distributions.normal import Normal
import numpy as np
from utils import *
from plots import *
from objectives import *

In [ ]:
## training parameters
STEPS = 10000
NUM_SAMPLES = 5
NUM_SAMPLES_SNR = 100
LEARNING_RATE = 5*1e-3
## model parameters
p_mu = torch.tensor([0.0])
p_sigma = torch.tensor([1.0])
q_mu = torch.tensor([4.0], requires_grad=True)
q_sigma = torch.tensor([2.0], requires_grad=True) 
## initialize optimizer
optimizer = torch.optim.SGD([q_mu, q_sigma], lr=LEARNING_RATE)

In [ ]:
EUBOs, ELBOs, IWELBOs, ESSs, SNRs, VARs = train(dreg, q_mu, q_sigma, p_mu, p_sigma, STEPS, NUM_SAMPLES, NUM_SAMPLES_SNR, optimizer, filename='rws', alpha=0.0)

In [ ]:
def plot_results(EUBO, ELBO, ESS, num_samples, snr_mu, snr_sigma):
    fig = plt.figure(figsize=(10,10))
    ax1, ax2, ax3 = fig.subplots(3, 1, sharex=True)
    plt.tight_layout()
    ax1.plot(EUBO, 'r', label='EUBOs')
    ax1.plot(ELBO, 'b', label='ELBOs')
    ax1.legend()
    ## SNR
    ax2.set_yscale('log')
    ax2.plot(snr_sigma, label='SNR_sigma')
    ax2.plot(snr_mu, label='SNR_mu')
    ax2.legend()
    ax2.set_ylim([1e-2,1e2])
    ## ESS
    ess_ratio = np.array(ESS) / num_samples
    ave_ess = np.reshape(ess_ratio, (-1, 10)).mean(-1)
    N = ave_ess.shape[0]
    ax3.plot(np.arange(N) * 10, ave_ess, 'go', label='ESS')
    ax3.set_ylim([0, 1])
# plot_results(EUBO, ELBO, ESS, num_samples, snr_mu, snr_sigma)